# The TensorFlow way

### © Jubeen Shah 2018

Hey there! Welcome to `J.S Codes` jupyter notebooks for TensorFlow!
<br>

In [14]:
import tensorflow as tf
import numpy as np
sess = tf.Session()

In this notebook we'll learn how to connect and work with various layers, and also build some custom ones. The data we will generate and use will be representative of small random images, it is best to understand these types of operations on a simple example and how we can use some built in layers to perform calculations.

 - First lets create our sample 2D image, which is 4px X 4px in size. We will create it in 4 dimensions, them being, the image number, height, width and the channel.

In [15]:
x_shape = [1, 4, 4, 1]
x_val = np.random.uniform(size= x_shape)

 - Now let's create a placeholder in our graph where we can feed in the sample image, as follows

In [16]:
x_data = tf.placeholder(tf.float32, shape= x_shape)

 - To create a moving window to average across the 4x4 image, we will use a built in function that will convolute a constant across a window of the shape 2x2. This function is quite common in image processing, and the function we would be using is `conv2d()`. This function would take a piecewise product of the window and a filter we specify. We must also specify a `stride` for moving the window in both the diractions. Here, we will compute four moving averages, top left, top right, bottom left, bottom right four pixels. we do this by creating a 2x2 window having a stride of length 2 in each directioon. to take the average, we will convolute the 2x2 window with a constant of 0.25

In [17]:
my_filter = tf.constant(0.25, shape= [2,2,1,1])
my_strides = [1,2,2,1]
mov_avg_lyr = tf.nn.conv2d(input=x_data,filter=my_filter, strides=my_strides, padding='SAME', name='moving_avg_window')

To figure out the output size of a convolutional layer we can use the following formula,



$Output = \frac {W - F + 2P}{ S + 1}$

where, <br>
W - Window Size, <br>
F - Filter Size, <br>
P - Padding of Zeros, and<br>
s - Stride <br>



 - Let us now define a custom layer that will operate on the 2x2 output of the moving average window. The custom function will first multiply the input by another 2x2 matrix sensor, and them add one to each entry. After this, we take the sigmoid of each element and return the 2x2 matric. Since matrix multiplication only operates on two-dimensional matrices, we need to drop the extra dimension of our image that are of size 1. TensorFlow can do this with the built in function `tf.squeeze()`

In [18]:
def custom_layer(input_matrix):
    input_matrix_squeezed = tf.squeeze(input_matrix)
    A = tf.constant([[1.,2.],[-1.,3.,]])
    b = tf.constant(1., shape=[2,2])
    temp1 = tf.matmul(A, input_matrix_squeezed)
    temp = tf.add(temp1, b)
    return(tf.sigmoid(temp))

 - Now, we place the new layer on the graph. We will do this with the help of named scope so that it identifiable and collapsabe/ expandable on the computational graph.

In [19]:
with tf.name_scope('Custom_Layer') as scope:
    custom_layer1 = custom_layer(mov_avg_lyr)

 - No we just feed in4x4 image in the placeholder and tell tensorFlow to run the graph.

In [20]:
 print(sess.run(custom_layer1, feed_dict={x_data : x_val}))

[[0.9464942  0.9147585 ]
 [0.90335053 0.91176784]]
